In [1]:
!pip install --upgrade ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
!rm -rf /data/runs/*

In [3]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
from torch.utils.tensorboard import SummaryWriter

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [4]:
SEED=1

In [5]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [6]:
torch.cuda.device_count()

1

In [7]:
START_EPOCH = 0

In [8]:
ARCH = 'resnet152'
EPOCHS = 5
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=256
VAL_BATCH=256
WORKERS=2
TRAINDIR="/data/CINIC/train"
VALDIR="/data/CINIC/valid"
GPU="cuda:0"

In [9]:
# set your active device to your GPU in this cell
device = 'cuda'

In [10]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [11]:
#torch.cuda.set_device(GPU)

In [12]:
cudnn.benchmark = True

In [13]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    global_step = 0
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

#         if GPU is not None:
#             images = images.cuda(GPU, non_blocking=True)
#         if torch.cuda.is_available():
#             target = target.cuda(GPU, non_blocking=True)
        images = images.to(device)
        target = target.to(device)

        # Enables autocasting for the forward pass (model + loss)
        with torch.autocast(device_type=device):
            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            writer.add_scalar("accuracy/acc1", acc1, global_step)
            writer.add_scalar("accuracy/acc5", acc5, global_step)
            writer.add_scalar("loss", loss, global_step)
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
            global_step += 1

            if i % PRINT_FREQ == 0:
                progress.display(i)
                


In [14]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
#             if GPU is not None:
#                 images = images.cuda(GPU, non_blocking=True)
#             if torch.cuda.is_available():
#                 target = target.cuda(GPU, non_blocking=True)

            images = images.to(device)
            target = target.to(device)

            # compute output
            with torch.autocast(device_type=device):
                output = model(images)
                loss = criterion(output, target)

                # measure accuracy and record loss
                acc1, acc5 = accuracy(output, target, topk=(1, 5))
                losses.update(loss.item(), images.size(0))
                top1.update(acc1[0], images.size(0))
                top5.update(acc5[0], images.size(0))

                # measure elapsed time
                batch_time.update(time.time() - end)
                end = time.time()

                if i % PRINT_FREQ == 0:
                    progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

In [15]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [16]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [17]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [18]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [19]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [20]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [21]:
normalize = transforms.Normalize(mean=cifar_mean_RGB, std=cifar_std_RGB)

In [22]:
IMG_SIZE = 32
# IMG_SIZE = 224

In [23]:
NUM_CLASSES = 10

In [24]:
#model = models.__dict__[ARCH]()
model = models.resnet152(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

In [26]:
inf = model.fc.in_features

In [27]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [28]:
#model.cuda(GPU)
model = model.to(device)

In [29]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [30]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [31]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [32]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
])

In [33]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [34]:
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
])

In [35]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [36]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [37]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [38]:
best_acc1 = 0

In [39]:
# Create a SummaryWriter to log accuracy, loss, and learning rate during training
writer = SummaryWriter(log_dir="/data/runs/")

for epoch in range(START_EPOCH, EPOCHS):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)
    
    # Check how much GPU memory was used during training
    allocated_memory = torch.cuda.max_memory_allocated(device=device)
    total_memory = torch.cuda.get_device_properties(device).total_memory
    print(f"Batch size: {TRAIN_BATCH}, allocated memory: {allocated_memory / 1024 / 1024} MB total memory: {total_memory / 1024 / 1024} MB")

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
# Close the SummaryWriter
writer.close()

Epoch: [0][  0/352]	Time  1.364 ( 1.364)	Data  0.190 ( 0.190)	Loss 2.3803e+00 (2.3803e+00)	Acc@1  12.89 ( 12.89)	Acc@5  51.17 ( 51.17)
Epoch: [0][ 50/352]	Time  0.071 ( 0.096)	Data  0.000 ( 0.004)	Loss 2.4355e+00 (3.2169e+00)	Acc@1  11.72 ( 13.47)	Acc@5  60.94 ( 56.31)
Epoch: [0][100/352]	Time  0.071 ( 0.085)	Data  0.000 ( 0.002)	Loss 2.3710e+00 (2.8160e+00)	Acc@1   8.98 ( 14.43)	Acc@5  42.19 ( 59.57)
Epoch: [0][150/352]	Time  0.070 ( 0.080)	Data  0.000 ( 0.002)	Loss 2.2347e+00 (2.6675e+00)	Acc@1  16.80 ( 13.95)	Acc@5  66.41 ( 59.41)
Epoch: [0][200/352]	Time  0.070 ( 0.078)	Data  0.000 ( 0.001)	Loss 2.1854e+00 (2.5648e+00)	Acc@1  18.75 ( 14.97)	Acc@5  66.41 ( 61.77)
Epoch: [0][250/352]	Time  0.070 ( 0.076)	Data  0.000 ( 0.001)	Loss 2.2017e+00 (2.4889e+00)	Acc@1  20.31 ( 15.99)	Acc@5  71.48 ( 63.95)
Epoch: [0][300/352]	Time  0.071 ( 0.075)	Data  0.000 ( 0.001)	Loss 2.0561e+00 (2.4259e+00)	Acc@1  21.09 ( 16.82)	Acc@5  78.52 ( 65.95)
Epoch: [0][350/352]	Time  0.067 ( 0.075)	Data  0.000 ( 

In [40]:
# the logdir should match the parameter passed to what you pass to SummaryWriter, e.g.
# writer = SummaryWriter(log_dir="/data/runs/")
!tensorboard --logdir=/data/runs

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.9.0 at http://ip-172-31-5-57.us-west-2.compute.internal:6006/ (Press CTRL+C to quit)
